In [1]:
# import os

# # 设置环境变量
# os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# # 打印环境变量以确认设置成功
# print(os.environ.get('HF_ENDPOINT'))

import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import Trainer
import evaluate
import numpy as np
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
import json
from transformers import set_seed
import random
import numpy as np
import torch
from tqdm import tqdm

2025-12-26 15:53:13.358209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-26 15:53:13.373769: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-26 15:53:13.378501: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-26 15:53:13.389950: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-12-26 15:53:14.291423: W tensorflow/compiler/tf2

In [3]:
# 2. 设置随机种子
# 这对于"无微调"模型尤为重要，因为分类头(Classification Head)是随机初始化的
# 不同的种子会导致完全不同的随机猜测结果

# 动态生成随机种子
import random
#seed = random.randint(0, 10000)
seed = 7967
set_seed(seed)

result = {}
result["seed"] = seed
result["type"] = "no_finetune_baseline" # 标记这是无微调的基线测试

print(f"Running Baseline Test (No Training) with Seed: {seed}")

Running Baseline Test (No Training) with Seed: 7967


In [4]:
# 3. 初始化模型和分词器
model_checkpoint = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.pad_token = tokenizer.eos_token

# 加载模型：注意，这里加载的是预训练的 Transformer + 随机初始化的分类头
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model.config.pad_token_id = model.config.eos_token_id

# 确保模型在 GPU 上
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval() # 直接进入评估模式

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

In [5]:
# 定义分词函数
def tokenize_function(example):
    return tokenizer(
        example["sentence1"], 
        example["sentence2"], 
        truncation=True,
        max_length=256, 
        padding="max_length"
    )

# 定义推理函数 (封装以复用)
def run_inference(test_dataset, batch_size=64):
    preds = []
    labels = []
    
    for i in tqdm(range(0, len(test_dataset), batch_size), desc="Predicting"):
        batch = test_dataset[i : i + batch_size]
        
        # 转换为 Tensor 并移动到 GPU
        inputs = {
            "input_ids": torch.tensor(batch["input_ids"]).to(device),
            "attention_mask": torch.tensor(batch["attention_mask"]).to(device),
        }
        batch_labels = batch["label"] 

        with torch.no_grad():
            outputs = model(**inputs)
            # 取最大概率的类别
            batch_preds = torch.argmax(outputs.logits, axis=-1).cpu().numpy() 

        preds.extend(batch_preds)
        labels.extend(batch_labels)
        
    metric = evaluate.load("glue", "mrpc")
    return metric.compute(predictions=preds, references=labels)

In [7]:
# ==========================================
# 测试集 1: protein_sim_pair_450bp (全长/较长序列)
# ==========================================
print("Loading Test Dataset 1: protein_sim_pair_450bp...")
raw_datasets_1 = load_dataset('dnagpt/gene_lan_transfer', 'protein_sim_pair_450bp')['train'].train_test_split(test_size=0.1, seed=seed)
    
# 你的原始逻辑中，这个数据集似乎不需要翻转标签，只是为了过一下 map 流程
# 保持原样以确保数据一致性
def process_data_1(example):
    return example

processed_datasets_1 = raw_datasets_1.map(process_data_1, batched=False)
tokenized_datasets_1 = processed_datasets_1.map(tokenize_function, batched=True, num_proc=4)

# 运行推理
ret_1 = run_inference(tokenized_datasets_1["test"])
result["dna_protein_pair_full"] = ret_1
print(f"Result 1: {ret_1}")

Loading Test Dataset 1: protein_sim_pair_450bp...


Map:   0%|          | 0/16200 [00:00<?, ? examples/s]

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/16200 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1800 [00:00<?, ? examples/s]

Predicting: 100%|██████████| 29/29 [00:04<00:00,  7.22it/s]


Result 1: {'accuracy': 0.55, 'f1': 0.14915966386554622}


In [10]:
# ==========================================
# 测试集 2: protein_sim_pair_150bp (较短/随机翻转)
# ==========================================

raw_datasets_2 = load_dataset('dnagpt/gene_lan_transfer', 'protein_sim_pair_150bp')['train'].train_test_split(test_size=0.5, seed=seed)

# 你的原始逻辑中，这里进行了标签翻转 (label = 1 - label)
def flip_labels(example):
    return example

flipped_datasets_2 = raw_datasets_2.map(flip_labels, batched=False)
tokenized_datasets_2 = flipped_datasets_2.map(tokenize_function, batched=True, num_proc=4)

# 运行推理
ret_2 = run_inference(tokenized_datasets_2["test"])
result["protein_pair_50"] = ret_2
print(f"Result 2: {ret_2}")



# ==========================================
# 输出结果
# ==========================================
print(json.dumps(result))

Loading Test Dataset 2: protein_sim_pair_150bp...


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/9000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/9000 [00:00<?, ? examples/s]

Predicting: 100%|██████████| 141/141 [00:19<00:00,  7.35it/s]


Result 2: {'accuracy': 0.5484444444444444, 'f1': 0.16378600823045267}
{"seed": 7967, "type": "no_finetune_baseline", "dna_protein_pair_full": {"accuracy": 0.55, "f1": 0.14915966386554622}, "protein_pair_50": {"accuracy": 0.5484444444444444, "f1": 0.16378600823045267}}
